<a href="https://colab.research.google.com/github/muhdlaziem/DR/blob/master/mergeLevel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [23]:
%cd 'My Drive'/diabetic-retinopathy-resized/

/gdrive/My Drive/diabetic-retinopathy-resized


In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import cv2

In [0]:
DR = pd.read_csv('trainLabels_cropped.csv')
DR['image'] = DR['image'].apply(lambda x:'./resized_train_cropped/resized_train_cropped/' + x + '.jpeg')
DR = DR.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1)

In [26]:
DR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35108 entries, 0 to 35107
Data columns (total 2 columns):
image    35108 non-null object
level    35108 non-null int64
dtypes: int64(1), object(1)
memory usage: 548.7+ KB


In [27]:
(DR['level'].value_counts())/len(DR)

0    0.734932
2    0.150621
1    0.069443
3    0.024838
4    0.020166
Name: level, dtype: float64

In [28]:
DR['level(3)'] = DR['level']
DR['level(3)'] = DR['level(3)'].replace({2:1})
DR['level(3)'] = DR['level(3)'].replace({3:2})
DR['level(3)'] = DR['level(3)'].replace({4:2})

(DR['level(3)'].value_counts())/len(DR)

0    0.734932
1    0.220064
2    0.045004
Name: level(3), dtype: float64

In [29]:
DR['level(2)'] = DR['level']
DR['level(2)'] = DR['level(2)'].replace({2:1})
DR['level(2)'] = DR['level(2)'].replace({3:1})
DR['level(2)'] = DR['level(2)'].replace({4:1})
(DR['level(2)'].value_counts())/len(DR)

0    0.734932
1    0.265068
Name: level(2), dtype: float64

In [30]:
DR.head()

,image,level,level(3),level(2)
0,./resized_train_cropped/resized_train_cropped/...,0,0,0
1,./resized_train_cropped/resized_train_cropped/...,0,0,0
2,./resized_train_cropped/resized_train_cropped/...,0,0,0
3,./resized_train_cropped/resized_train_cropped/...,0,0,0
4,./resized_train_cropped/resized_train_cropped/...,1,1,1


In [0]:
# take index of each level
level_0_idx = DR['level(3)']==0
level_1_idx = DR['level(3)']==1
level_2_idx = DR['level(3)']==2


In [32]:
# form a dataframe
level_0_df = DR[level_0_idx]
level_1_df = DR[level_1_idx]
level_2_df = DR[level_2_idx]

print(level_0_df.info())
print(level_1_df.info())
print(level_2_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25802 entries, 0 to 35106
Data columns (total 4 columns):
image       25802 non-null object
level       25802 non-null int64
level(3)    25802 non-null int64
level(2)    25802 non-null int64
dtypes: int64(3), object(1)
memory usage: 1007.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7726 entries, 4 to 35107
Data columns (total 4 columns):
image       7726 non-null object
level       7726 non-null int64
level(3)    7726 non-null int64
level(2)    7726 non-null int64
dtypes: int64(3), object(1)
memory usage: 301.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1580 entries, 6 to 35031
Data columns (total 4 columns):
image       1580 non-null object
level       1580 non-null int64
level(3)    1580 non-null int64
level(2)    1580 non-null int64
dtypes: int64(3), object(1)
memory usage: 61.7+ KB
None


In [0]:
#exclude 10% of level 0, 5% of level 1, and 5% of level 2
level_0_sample = level_0_df.sample(frac=0.1)
level_1_sample = level_1_df.sample(frac=0.05)
level_2_sample = level_2_df.sample(frac=0.05)

In [34]:
#drop sample indexes
DR = DR.drop(level_0_sample.index)
DR = DR.drop(level_1_sample.index)
DR = DR.drop(level_2_sample.index)
DR.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32063 entries, 0 to 35107
Data columns (total 4 columns):
image       32063 non-null object
level       32063 non-null int64
level(3)    32063 non-null int64
level(2)    32063 non-null int64
dtypes: int64(3), object(1)
memory usage: 1.2+ MB


In [0]:
# save sample
level_0_sample.to_csv('level_0_sample.csv',index= False)
level_1_sample.to_csv('level_1_sample.csv',index= False)
level_2_sample.to_csv('level_2_sample.csv',index= False)

In [0]:
# save sample
DR.to_csv('omitValidationData.csv',index= False)

In [0]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(DR['image'],DR['level(2)'],test_size=0.20, random_state=42)
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(DR['image'],DR['level(3)'],test_size=0.20, random_state=42)

In [0]:
def loadImage(df):
    images = list()
    for i in range(len(df)):
            image_name = df.iloc[i]
            # print(image_name)
            img = cv2.imread(image_name)
            res = cv2.resize(img, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
            images.append(res)
    return images

In [0]:
trainImages_2 = loadImage(X_train_2)
testImages_2 = loadImage(X_test_2)

trainImages_2 = np.asarray(trainImages_2)
testImages_2 = np.asarray(testImages_2)

print(trainImages_2.shape)
print(testImages_2.shape)

y_train_2 = to_categorical(y_train_2)
y_test_2 = to_categorical(y_test_2)
np.save('trainImages_2.npy', trainImages_2)
np.save('testImages_2.npy', testImages_2)
np.save('y_train_2.npy', y_train_2)
np.save('y_test_2.npy', y_test_2)


In [0]:
# X_train_2[0]

In [0]:
# !ls

In [0]:
# img = cv2.imread(X_train_2[0])

In [0]:
# print(img.shape)

In [0]:
# cv2.resize(img, dsize=(256, 256), interpolation=cv2.INTER_CUBIC).shape